# LaBr3 波形解堆积算法

### 数据

- 每轮约 20 万个长波形，每个波形 20000 个点

### 触发逻辑

- 快速滤波 (fastfilter) 过阈触发
   - 前 1000 个点无信号，用于评估噪声水平
   - 取噪声水平的若干倍为阈值
   
### 标准波形

- 筛选无堆积的波形
   - 粗筛：触发前后一定范围不存在其他触发（指快速滤波触发）
   - 细筛：计算波形半高宽，画出峰宽-峰高的二维关联图，用 TCutG 选取密集条带区域，该区域为良好的无堆积事件
   - p.s. 条带外的散点来自非常紧密的堆积事件，它们无法用快速滤波分辨出来
- 求解波形形状随波形高度的变化关系
   - 计算波形最高点 xmax ymax
   - 计算最高点 xmax 附近某点 x0 处的相对高度 y0/ymax
   - 对于固定的 x0，画出 y0/ymax : ymax 二维关联图，用二次函数拟合
   - 这样，只要给定 ymax，就能画出相应的标准波形

### 波形解堆积与拟合

- 将原始长波形分段，堆积的信号一起拟合，无堆积的信号相互分离
- 以触发位置为初值，进行第一次拟合
- 计算拟合结果与原波形的残差，对残差图寻峰，在峰值处尝试添加新峰拟合
   - 添加新峰，要求残差大于噪声水平，且新峰拟合后的位置不能与已有峰重合，且必须将 chi2/ndf 降低到一定比例以下
   - 若以上条件不满足，则选取残差的另一峰值尝试
   - 若确定添加新峰，则再次计算拟合残差，重复本步骤
   - 若所有残差峰值处都不能添加新峰，则结束循环
- 最后再次检查并剔除位置重合的峰（添加新峰可能会影响原有峰位）

### 模拟

- 按照标准波形生成模拟信号，包括波形高度与波形积分的关联关系
- 按照事件间隔的泊松分布安排事件的发生时间：$$I(t)=re^{-rt}dt$$其中 r 为事件计数率，其单位应与抽样得到的 t 的单位相同。
- 设置阈值，将模拟生成的超出阈值的信号设置为阈值
- 按照实验情况设置基线的高斯白噪声

In [1]:
!date

2023年 11月 17日 星期五 11:26:54 CST

